# Initialization

In [1]:
from __future__ import print_function


from keras.models import Sequential, Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard,ReduceLROnPlateau
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
from keras.layers import AveragePooling2D, Multiply, Concatenate, Add

import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import pickle

import os, cv2
from preprocessing import BatchGenerator
from utils import decode_netout, draw_boxes

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

from keras.regularizers import l2
from keras.models import Model
from keras.applications.mobilenet import MobileNet
from keras.layers.merge import concatenate

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os


Using TensorFlow backend.


In [2]:
df_test          = pd.read_csv('data/level3_data/test.csv')
df               = pd.read_csv('data/level3_data/training_set.csv')

train_dir        = 'data/train/images/'
test_dir         = 'data/train/images/'
LABELS           = ['obj']
BOX              = 10
CLASS            = len(LABELS)
CLASS_WEIGHTS    = np.ones(CLASS, dtype='float32')
OBJ_THRESHOLD    = 0.3
NMS_THRESHOLD    = 0.45
ANCHORS          = [1.45,5.42, 2.03,2.28, 2.59,5.14, 3.87,2.99, 4.29,4.23, 4.30,5.98, 5.00,1.98, 5.92,3.04, 6.19,4.52, 6.29,6.20]
IMAGE_H, IMAGE_W = 224, 224
GRID_H,  GRID_W  = 7, 7
NO_OBJECT_SCALE  = 1.0
OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
CLASS_SCALE      = 1.0
BATCH_SIZE       = 128
WARM_UP_BATCHES  = 0
TRUE_BOX_BUFFER  = 10#0

In [ ]:
import random
import argparse
import numpy as np

# from preprocessing import parse_annotation
import json

argparser = argparse.ArgumentParser()

argparser.add_argument(
    '-c',
    '--conf',
    default='config.json',
    help='path to configuration file')

argparser.add_argument(
    '-a',
    '--anchors',
    default=5,
    help='number of anchors to use')

def IOU(ann, centroids):
    w, h = ann
    similarities = []

    for centroid in centroids:
        c_w, c_h = centroid

        if c_w >= w and c_h >= h:
            similarity = w*h/(c_w*c_h)
        elif c_w >= w and c_h <= h:
            similarity = w*c_h/(w*h + (c_w-w)*c_h)
        elif c_w <= w and c_h >= h:
            similarity = c_w*h/(w*h + c_w*(c_h-h))
        else: #means both w,h are bigger than c_w and c_h respectively
            similarity = (c_w*c_h)/(w*h)
        similarities.append(similarity) # will become (k,) shape

    return np.array(similarities)

def avg_IOU(anns, centroids):
    n,d = anns.shape
    sum = 0.

    for i in range(anns.shape[0]):
        sum+= max(IOU(anns[i], centroids))

    return sum/n

def print_anchors(centroids):
    anchors = centroids.copy()

    widths = anchors[:, 0]
    sorted_indices = np.argsort(widths)

    r = "anchors: ["
    for i in sorted_indices[:-1]:
        r += '%0.2f,%0.2f, ' % (anchors[i,0], anchors[i,1])

    #there should not be comma after last anchor, that's why
    r += '%0.2f,%0.2f' % (anchors[sorted_indices[-1:],0], anchors[sorted_indices[-1:],1])
    r += "]"

    print(r)
    return r

def run_kmeans(ann_dims, anchor_num):
    ann_num = ann_dims.shape[0]
    iterations = 0
    prev_assignments = np.ones(ann_num)*(-1)
    iteration = 0
    old_distances = np.zeros((ann_num, anchor_num))

    indices = [random.randrange(ann_dims.shape[0]) for i in range(anchor_num)]
    centroids = ann_dims[indices]
    anchor_dim = ann_dims.shape[1]

    while True:
        distances = []
        iteration += 1
        for i in range(ann_num):
            d = 1 - IOU(ann_dims[i], centroids)
            distances.append(d)
        distances = np.array(distances) # distances.shape = (ann_num, anchor_num)

        print("iteration {}: dists = {}".format(iteration, np.sum(np.abs(old_distances-distances))))

        #assign samples to centroids
        assignments = np.argmin(distances,axis=1)

        if (assignments == prev_assignments).all() :
            return centroids

        #calculate new centroids
        centroid_sums=np.zeros((anchor_num, anchor_dim), np.float)
        for i in range(ann_num):
            centroid_sums[assignments[i]]+=ann_dims[i]
        for j in range(anchor_num):
            centroids[j] = centroid_sums[j]/(np.sum(assignments==j) + 1e-6)

        prev_assignments = assignments.copy()
        old_distances = distances.copy()
       

    
    
train_data_prepare = []

for i in range(df.shape[0]):
    col = df.iloc[i]
    one_file = {}
    one_file['filename'] = train_dir + col['image_name']
    one_file['width'] = 640
    one_file['height'] = 480

    one_file['object'] = []
    inner_object = {}
    inner_object['name'] = 'obj'
    inner_object['xmin'] = col['x1']
    inner_object['ymin'] = col['y1']
    inner_object['xmax'] = col['x2']
    inner_object['ymax'] = col['y2']
    
    
    one_file['object'] = [inner_object]
    train_data_prepare.append(one_file)
    
grid_h = GRID_H
grid_w = GRID_W
# run k_mean to find the anchors
annotation_dims = []
for image in train_data_prepare:
    cell_w = image['width']/grid_w
    cell_h = image['height']/grid_h
    
    for obj in image['object']:
        relative_w = (float(obj['xmax']) - float(obj['xmin']))/cell_w
        relatice_h = (float(obj["ymax"]) - float(obj['ymin']))/cell_h
        annotation_dims.append(tuple(map(float, (relative_w,relatice_h))))

annotation_dims = np.array(annotation_dims)


def print_anchors(centroids):
    anchors = centroids.copy()

    widths = anchors[:, 0]
    sorted_indices = np.argsort(widths)

    r = "anchors: ["
    for i in sorted_indices[:-1]:
        r += '%0.2f,%0.2f, ' % (anchors[i,0], anchors[i,1])

    #there should not be comma after last anchor, that's why
    r += '%0.2f,%0.2f' % (anchors[sorted_indices[-1:],0], anchors[sorted_indices[-1:],1])
    r += "]"

    print(r)
    return r

    
num_anchors = 10
centroids = run_kmeans(annotation_dims, num_anchors)

# write anchors to file
print('\naverage IOU for', num_anchors, 'anchors:', '%0.2f' % avg_IOU(annotation_dims, centroids))
ANCHORS = print_anchors(centroids)

# Put this printed anchor in the Anchor place

In [3]:
class BaseFeatureExtractor(object):
    """docstring for ClassName"""

    # to be defined in each subclass
    def __init__(self, input_size):
        raise NotImplementedError("error message")

    # to be defined in each subclass
    def normalize(self, image):
        raise NotImplementedError("error message")       

    def get_output_shape(self):
        return self.feature_extractor.get_output_shape_at(-1)[1:3]

    def extract(self, input_image):
        return self.feature_extractor(input_image)

    
class MobileNetFeature(BaseFeatureExtractor):
    """docstring for ClassName"""
    def __init__(self, input_size):
        input_image = Input(shape=(input_size, input_size, 3))

        mobilenet = MobileNet(input_shape=(224,224,3), include_top=False, weights=None)
        x = mobilenet(input_image)
        self.feature_extractor = Model(input_image, x)  

    def normalize(self, image):
        image = image / 255.
        image = image - 0.5
        image = image * 2.

        return image
 

In [4]:
input_size = 224
max_box_per_image = TRUE_BOX_BUFFER # ANCHORS * BOX
nb_box = BOX
nb_class = CLASS
    
input_image     = Input(shape=(input_size, input_size, 3), name='input_img')
true_boxes = Input(shape=(1, 1, 1, max_box_per_image , 4))  


def build_model(input_image, true_boxes):
    input_size = 224
    max_box_per_image = TRUE_BOX_BUFFER # ANCHORS * BOX
    nb_box = BOX
    nb_class = CLASS

    feature_extractor = MobileNetFeature(input_size)

    # print(feature_extractor.get_output_shape())    
    grid_h, grid_w = feature_extractor.get_output_shape()        
    features = feature_extractor.extract(input_image)            
    ##################################################################

    block_res1 = features

    features_ = BatchNormalization()(block_res1)
    features_ = Activation('relu')(features_)
    features_ = Conv2D(256, (1, 1), 
                       padding='same',
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(1e-4))(features_)
    features_ = BatchNormalization()(features_)
    features_ = Activation('relu')(features_)

    features_ = Conv2D(256, (3, 3), 
                       padding='same',
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(1e-4))(features_)
    features_ = BatchNormalization()(features_)
    features_ = Activation('relu')(features_)
    #################################

    block_res2 = features_
    features_ = Conv2D(1024, (1, 1), 
                       padding='same',
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(1e-4))(features_)

    #################################
    features_ = Add()([block_res1, features_])

    features_ = Conv2D(256, (1, 1), 
                       padding='same',
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(1e-4))(features_)
    features_ = BatchNormalization()(features_)
    features_ = Activation('relu')(features_)

    features_ = Conv2D(256, (3, 3), 
                       padding='same',
                       kernel_initializer='he_normal',
                       kernel_regularizer=l2(1e-4))(features_)
    #################################

    new_block1 = features_
    features_ = BatchNormalization()(features_)
    features_ = Activation('relu')(features_)
    new_block2 = features_
    #################################

    
    #################################
    # channel attention

    c_global_feat = AveragePooling2D(pool_size=(7, 7))(new_block2)
    c_global_feat = Conv2D(256, (1, 1), 
                           padding='same',
                           kernel_initializer='he_normal',
                           kernel_regularizer=l2(1e-4))(c_global_feat)
    #     c_global_feat = BatchNormalization()(c_global_feat)
    c_global_feat = Activation('relu')(c_global_feat)

    c_global_feat = Conv2D(256, (1, 1), 
                           padding='same',
                           kernel_initializer='he_normal',
                           kernel_regularizer=l2(1e-4))(c_global_feat)
    c_global_feat = Activation('sigmoid')(c_global_feat)

    channel_weighted_feat = Multiply()([new_block2, c_global_feat])
    channel_weighted_feat = Conv2D(256, (1, 1), 
                                   padding='same',
                                   kernel_initializer='he_normal',
                                   kernel_regularizer=l2(1e-4))(channel_weighted_feat)
    #################################


    #################################
    # spatial attention

    s_global_feat = Conv2D(256, (1, 1), 
                           padding='same',
                           kernel_initializer='he_normal',
                           kernel_regularizer=l2(1e-4))(new_block2)
    s_global_feat = Activation('relu')(s_global_feat)

    s_global_feat = Conv2D(1, (1, 1), 
                           padding='same',
                           kernel_initializer='he_normal',
                           kernel_regularizer=l2(1e-4))(s_global_feat)
    s_global_feat = Activation('sigmoid')(s_global_feat)

    spatial_weighted_feat = Multiply()([new_block2, s_global_feat])
    spatial_weighted_feat = Conv2D(256, (1, 1), 
                                   padding='same',
                                   kernel_initializer='he_normal',
                                   kernel_regularizer=l2(1e-4))(spatial_weighted_feat)
    
    #################################


    #################################
    # concat both attention features

    concat_s_c_feat = Concatenate(axis=-1)([channel_weighted_feat, spatial_weighted_feat])
    concat_s_c_feat = Conv2D(256, (1, 1), 
                             padding='same',
                             kernel_initializer='he_normal',
                             kernel_regularizer=l2(1e-4))(concat_s_c_feat)

    final_feat = Add()([new_block1, concat_s_c_feat])

    #################################
    
    x = final_feat
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    ##################################################################
    # make the object detection layer
    output = Conv2D(nb_box * (4 + 1 + nb_class), 
                    (1,1), strides=(1,1), 
                    padding='same', 
                    name='DetectionLayer', 
                    kernel_initializer='lecun_normal')(x)#(features)
    output = Reshape((grid_h, grid_w, nb_box, 4 + 1 + nb_class))(output)

    model = Model([input_image, true_boxes], output)
    return model



In [5]:
def custom_loss(y_true, y_pred):
    mask_shape = tf.shape(y_true)[:4]
    
    cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(GRID_W), [GRID_H]), (1, GRID_H, GRID_W, 1, 1)))
    cell_y = tf.transpose(cell_x, (0,2,1,3,4))

    cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [BATCH_SIZE, 1, 1, BOX, 1])
    
    coord_mask = tf.zeros(mask_shape)
    conf_mask  = tf.zeros(mask_shape)
    class_mask = tf.zeros(mask_shape)
    
    seen = tf.Variable(0.)
    total_recall = tf.Variable(0.)
    
    """
    Adjust prediction
    """
    ### adjust x and y      
    pred_box_xy = tf.sigmoid(y_pred[..., :2]) + cell_grid
    
    ### adjust w and h
    pred_box_wh = tf.exp(y_pred[..., 2:4]) * np.reshape(ANCHORS, [1,1,1,BOX,2])
    
    ### adjust confidence
    pred_box_conf = tf.sigmoid(y_pred[..., 4])
    
#     ### adjust class probabilities
    pred_box_class = y_pred[..., 5:]
    
    """
    Adjust ground truth
    """
    ### adjust x and y
    true_box_xy = y_true[..., 0:2] # relative position to the containing cell
    
    ### adjust w and h
    true_box_wh = y_true[..., 2:4] # number of cells accross, horizontally and vertically
    
    ### adjust confidence
    true_wh_half = true_box_wh / 2.
    true_mins    = true_box_xy - true_wh_half
    true_maxes   = true_box_xy + true_wh_half
    
    pred_wh_half = pred_box_wh / 2.
    pred_mins    = pred_box_xy - pred_wh_half
    pred_maxes   = pred_box_xy + pred_wh_half       
    
    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
    
    true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
    pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)
    
    true_box_conf = iou_scores * y_true[..., 4]
    
#     ### adjust class probabilities
    true_box_class = tf.argmax(y_true[..., 5:], -1)
    
    """
    Determine the masks
    """
    ### coordinate mask: simply the position of the ground truth boxes (the predictors)
    coord_mask = tf.expand_dims(y_true[..., 4], axis=-1) * COORD_SCALE
    
    ### confidence mask: penelize predictors + penalize boxes with low IOU
    # penalize the confidence of the boxes, which have IOU with some ground truth box < 0.6
    true_xy = true_boxes[..., 0:2]
    true_wh = true_boxes[..., 2:4]
    
    true_wh_half = true_wh / 2.
    true_mins    = true_xy - true_wh_half
    true_maxes   = true_xy + true_wh_half
    
    pred_xy = tf.expand_dims(pred_box_xy, 4)
    pred_wh = tf.expand_dims(pred_box_wh, 4)
    
    pred_wh_half = pred_wh / 2.
    pred_mins    = pred_xy - pred_wh_half
    pred_maxes   = pred_xy + pred_wh_half    
    
    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
    
    true_areas = true_wh[..., 0] * true_wh[..., 1]
    pred_areas = pred_wh[..., 0] * pred_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)

    best_ious = tf.reduce_max(iou_scores, axis=4)
    conf_mask = conf_mask + tf.to_float(best_ious < 0.6) * (1 - y_true[..., 4]) * NO_OBJECT_SCALE
    
    # penalize the confidence of the boxes, which are reponsible for corresponding ground truth box
    conf_mask = conf_mask + y_true[..., 4] * OBJECT_SCALE
    
#     ### class mask: simply the position of the ground truth boxes (the predictors)
    class_mask = y_true[..., 4] * tf.gather(CLASS_WEIGHTS, true_box_class) * CLASS_SCALE       
    
    """
    Warm-up training
    """
    no_boxes_mask = tf.to_float(coord_mask < COORD_SCALE/2.)
    seen = tf.assign_add(seen, 1.)
    
    true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, WARM_UP_BATCHES), 
                          lambda: [true_box_xy + (0.5 + cell_grid) * no_boxes_mask, 
                                   true_box_wh + tf.ones_like(true_box_wh) * np.reshape(ANCHORS, [1,1,1,BOX,2]) * no_boxes_mask, 
                                   tf.ones_like(coord_mask)],
                          lambda: [true_box_xy, 
                                   true_box_wh,
                                   coord_mask])
    
    """
    Finalize the loss
    """
    nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
    nb_conf_box  = tf.reduce_sum(tf.to_float(conf_mask  > 0.0))
    nb_class_box = tf.reduce_sum(tf.to_float(class_mask > 0.0))
    
    loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_wh    = tf.reduce_sum(tf.square(true_box_wh-pred_box_wh)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_conf  = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf) * conf_mask)  / (nb_conf_box  + 1e-6) / 2.
    loss_class = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=true_box_class, logits=pred_box_class)
    loss_class = tf.reduce_sum(loss_class * class_mask) / (nb_class_box + 1e-6)
    
    loss = 3*loss_xy + 3*loss_wh + 2.5*loss_conf + 1.5*loss_class
    
    nb_true_box = tf.reduce_sum(y_true[..., 4])
    nb_pred_box = tf.reduce_sum(tf.to_float(true_box_conf > 0.5) * tf.to_float(pred_box_conf > 0.3))

    return loss

In [6]:
generator_config = {
    'IMAGE_H'         : IMAGE_H, 
    'IMAGE_W'         : IMAGE_W,
    'GRID_H'          : GRID_H,  
    'GRID_W'          : GRID_W,
    'BOX'             : BOX,
    'LABELS'          : LABELS,
    'CLASS'           : len(LABELS),
    'ANCHORS'         : ANCHORS,
    'BATCH_SIZE'      : BATCH_SIZE,
    'TRUE_BOX_BUFFER' : TRUE_BOX_BUFFER,
}

In [7]:
def normalize(image):
    return image / 255.

In [8]:
train_data_prepare = []

for i in range(df.shape[0]):
    col = df.iloc[i]
    one_file = {}
    one_file['filename'] = train_dir + col['image_name']
    one_file['width'] = 640
    one_file['height'] = 480

    one_file['object'] = []
    inner_object = {}
    inner_object['name'] = 'obj'
    inner_object['xmin'] = col['x1']
    inner_object['ymin'] = col['y1']
    inner_object['xmax'] = col['x2']
    inner_object['ymax'] = col['y2']
    
    
    one_file['object'] = [inner_object]
    train_data_prepare.append(one_file)

len(train_data_prepare)

24000

# Add lvel2 data if you want

In [10]:
# df_aagin = pd.read_csv('/home/ankish1/ankish_save/yolo/data/training.csv')

# train_dir = '/home/ankish1/ankish_save/yolo/data/train/images/'
# # train_data_prepare = []

# for i in range(df_aagin.shape[0]):
#     col = df_aagin.iloc[i]
#     one_file = {}
#     one_file['filename'] = train_dir + col['image_name']
#     one_file['width'] = 640
#     one_file['height'] = 480

#     one_file['object'] = []
#     inner_object = {}
#     inner_object['name'] = 'obj'
#     inner_object['xmin'] = col['x1']
#     inner_object['ymin'] = col['y1']
#     inner_object['xmax'] = col['x2']
#     inner_object['ymax'] = col['y2']
    
    
#     one_file['object'] = [inner_object]
#     train_data_prepare.append(one_file)

# len(train_data_prepare)

38000

# do split according

In [9]:
idx = np.random.permutation(len(train_data_prepare))
train_idx = idx[:22000] # do split according 
val_idx = idx[22000:]

train_imgs = [train_data_prepare[i] for i in train_idx]
seen_train_labels = {}
seen_train_labels['obj'] = len(train_imgs)

valid_imgs = [train_data_prepare[i] for i in val_idx]
seen_valid_labels = {}
seen_valid_labels['obj'] = len(valid_imgs)

len(train_imgs), len(valid_imgs), seen_train_labels, seen_valid_labels

(22000, 2000, {'obj': 22000}, {'obj': 2000})

In [10]:
train_batch = BatchGenerator(train_imgs, generator_config, norm=normalize)
valid_batch = BatchGenerator(valid_imgs, generator_config, norm=normalize, jitter=False)

# test dataset

In [ ]:
# temp = BatchGenerator(train_imgs, generator_config, norm=None)
# (i1, b1), g1 = temp.__getitem__(0)

# check available gpus

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

In [ ]:
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler


early_stop = EarlyStopping(monitor='val_loss', 
                           min_delta=0.001, 
                           patience=5, 
                           mode='min', 
                           verbose=1)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=2, 
                              min_lr=1e-8) 

LOG_FILE_PATH = 'attention-2nd-{epoch:02d}-{val_loss:.4f}.hdf5'

EPOCHS = 60


checkpoint = ModelCheckpoint(filepath=LOG_FILE_PATH, 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True)


import tensorflow as tf
from keras.utils import multi_gpu_model
from keras.optimizers import Adam


with tf.device('/cpu:0'):
    model = build_model(input_image, true_boxes)
    print("built model successfully")
    
    
optimizer = Adam(lr=0.5e-2, beta_1=0.9, beta_2=0.999)

parallel_model = multi_gpu_model(model, gpus=4)
parallel_model.compile(loss = custom_loss,
                       optimizer=optimizer)#,

EPOCHS = 100
print("let's start parallel model with 50 epochs")
parallel_model.fit_generator(generator        = train_batch, 
                             steps_per_epoch  = len(train_batch), 
                             epochs           = 100, 
                             verbose          = 1,
                             validation_data  = valid_batch,
                             validation_steps = len(valid_batch),
                             callbacks        = [reduce_lr, early_stop, checkpoint],
                             use_multiprocessing=True,
                             workers=4)



In [25]:
np.save('weights_last.npy', model.get_weights())

In [ ]:
ls1 = parallel_model.history.history['loss']
val_ls1 = parallel_model.history.history['val_loss']
lr1 = parallel_model.history.history['lr']


# increase lr_rate

In [ ]:
optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999)
parallel_model.compile(loss = custom_loss,
                       optimizer=optimizer)#,

print("let's start parallel model again for another 60 epochs")
parallel_model.fit_generator(generator        = train_batch, 
                             steps_per_epoch  = len(train_batch), 
                             epochs           = 60, 
                             verbose          = 1,
                             validation_data  = valid_batch,
                             validation_steps = len(valid_batch),
                             callbacks        = [early_stop, checkpoint, reduce_lr],
                             use_multiprocessing=True,
                             workers=4)


In [ ]:
ls2 = parallel_model.history.history['loss']
val_ls2 = parallel_model.history.history['val_loss']
lr2 = parallel_model.history.history['lr']

plt.plot(ls2)
plt.plot(val_ls2)

In [ ]:
optimizer = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999)
parallel_model.compile(loss = custom_loss,
                       optimizer=optimizer)#,

print("let's start parallel model again for another 50 epochs")
parallel_model.fit_generator(generator        = train_batch, 
                             steps_per_epoch  = len(train_batch), 
                             epochs           = 60, 
                             verbose          = 1,
                             validation_data  = valid_batch,
                             validation_steps = len(valid_batch),
                             callbacks        = [early_stop, checkpoint, reduce_lr],
                             use_multiprocessing=True,
                             workers=4)

In [ ]:
ls3 = parallel_model.history.history['loss']
val_ls3 = parallel_model.history.history['val_loss']
lr3 = parallel_model.history.history['lr']

In [ ]:
plt.plot(ls1+ls2+ls3)

In [ ]:
def predict(test_img, thresh=0.3):
#     thresh = 0.3
    image = cv2.imread(test_img)
    input_image = cv2.resize(image, (224, 224))
    input_image = input_image / 255.
    input_image = input_image[:,:,::-1]
    input_image = np.expand_dims(input_image, 0)

    dummy_array = np.zeros((1,1,1,1,TRUE_BOX_BUFFER,4))
    
    netout = parallel_model.predict([input_image, dummy_array])

    boxes = decode_netout(netout[0], 
                          obj_threshold=thresh,#OBJ_THRESHOLD,
                          nms_threshold=0.3,
                          anchors=ANCHORS, 
                          nb_class=CLASS)
    count = 1
    while not boxes:
        thresh = thresh -0.5*count
        boxes = decode_netout(netout[0], 
                              obj_threshold=thresh,#OBJ_THRESHOLD,
                              nms_threshold=0.3,
                              anchors=ANCHORS, 
                              nb_class=CLASS)
        count += 1
    if len(boxes)>1:
        score_ = boxes[0].get_score()
        index_  = 0
        for ii, bb in enumerate(boxes[1:]):
            score_1 = bb.get_score()
            if score_1>=score_:
                index_ = ii+1
                score_ = score_1
#         print(index_)
        return boxes[index_]
    else:
        return boxes


import pandas as pd
box_pred = []

# test_dir = 'data/train/images/'
# df_test = pd.read_csv('data/level3_data/test.csv')
all_files = df_test['image_name'].values
print(len(all_files))


for ii, file_ in enumerate(all_files):
    box_pred.append(predict(test_dir + file_, 0.4))
    if (ii%2000) == 0:
        print("done till {} images".format(ii))
        
empty_pred_idx = []
box_pred_values = []
count = 0
for ii, box_ in enumerate(box_pred):
    image_h, image_w = 480, 640
#     if not box_:
#         print(ii)
#         count += 1
# count
    if box_:
        try:
#             print(box_[0])
            xmin = int(box_[0].xmin * image_w)
            ymin = int(box_[0].ymin * image_h)
            xmax = int(box_[0].xmax * image_w)
            ymax = int(box_[0].ymax * image_h)
        except:
#             print(box_)
            xmin = int(box_.xmin * image_w)
            ymin = int(box_.ymin * image_h)
            xmax = int(box_.xmax * image_w)
            ymax = int(box_.ymax * image_h)

    else:
        empty_pred_idx.append(ii)
        xmin, ymin, xmax, ymax = 999, 999, 999, 999#0, 0, 640, 480
        print("====",ii,"====")
    box_pred_values.append((xmin, ymin, xmax, ymax))
    
    if ii%2000 == 0:
        print("reached to {} boxes".format(ii))
    
box_pred_values = np.array(box_pred_values)
box_pred_values[3], box_pred_values.shape


df_pred = pd.DataFrame(columns=['x1','y1','x2','y2'], data=box_pred_values)
df_img = pd.DataFrame(data=all_files, columns=['image_name'])
sub = pd.concat([df_img, df_pred['x1'], df_pred['x2'], df_pred['y1'], df_pred['y2']], axis=1)
sub.head()




sub.x1 = np.where(sub.x1<-1,-1,sub.x1)
sub.y1 = np.where(sub.y1<-1, -1, sub.y1)
sub.x2 = np.where(sub.x2>640, 640, sub.x2)
sub.y2 = np.where(sub.y2>480, 480, sub.y2)

sub.to_csv('final-submission.csv',index=None)

print("done with prediction step")


In [ ]:
sub.head(10)

In [ ]:
fig, ax = plt.subplots(4,6,figsize=(20,12))
axes = ax.flatten()

for i in range(24):
    idx_r = np.random.randint(sub.shape[0])
    # now
    image = cv2.imread(test_dir + all_files[idx_r])
    box__ = sub.iloc[idx_r]
    xmin, ymin, xmax, ymax = box__['x1'], box__['y1'], box__['x2'],box__['y2']

    image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (0,255,0), 3)
    axes[i].imshow(image[:,:,::-1])
plt.show()